In [3]:
from pathlib import Path
import json
import joblib
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [4]:
BASE            = Path.cwd().parents[0]  # or use Path("..").resolve() if notebook is in a subfolder
DATA_PROC       = BASE / "data" / "processed"
NPZ_ROOT        = DATA_PROC / "npz"
STATS_PATH      = DATA_PROC / "stats.json"
CHECKPOINT      = BASE / "logs" / "vae_experiment" / "version_2" / "checkpoints" / \
                  "vae-epoch=78-val_loss=0.0000.ckpt"

TRAIN_IDS_NPY   = DATA_PROC / "train.npy"
VAL_IDS_NPY     = DATA_PROC / "val.npy"

LAT_TRAIN_NPY   = DATA_PROC / "latent_train.npy"
LAT_VAL_NPY     = DATA_PROC / "latent_val.npy"         

MODEL_OUT       = BASE / "models" / "iso_model.pkl"
print(f"{BASE}")

c:\Users\David\___utad\4_Cuarto\TFG\proyecto_PLAsTiCC


In [5]:
THRESH_JSON     = BASE / "models" / "threshold_iso.json"

BATCH_SIZE      = 256
N_WORKERS       = 4
PROXY_RARE      = 2    

In [6]:
from data.dataset import LightCurveDataset
from train.model_vae import LitVAE

In [7]:
def extract_latent(ids_npy: Path, out_npy: Path) -> np.ndarray:
    """Pasa los object_id del .npy por el encoder y guarda la matriz µ."""
    if out_npy.exists():
        print(f"✓ {out_npy.name} ya existe → cargando")
        return np.load(out_npy)

    print(f"→ Extrayendo latentes para {out_npy.stem}")
    # Modelo
    model = LitVAE.load_from_checkpoint(str(CHECKPOINT))
    model.eval().cuda()
    latent_dim = model.hparams.latent_dim

    # Stats para Dataset
    stats = json.load(open(STATS_PATH))
    flux_mean = torch.tensor(stats["flux"]["mean"]).cuda()
    flux_std  = torch.tensor(stats["flux"]["std"]).cuda()
    meta_keys = list(stats["meta"].keys())
    meta_mean = torch.tensor([stats["meta"][k]["mean"] for k in meta_keys]).cuda()
    std_meta  = torch.tensor([stats["meta"][k]["std"]  for k in meta_keys]).cuda()

    ids = np.load(ids_npy)
    ds  = LightCurveDataset(ids, (flux_mean, flux_std, meta_mean, std_meta, meta_keys),
                            root=NPZ_ROOT)
    dl  = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                     num_workers=N_WORKERS, pin_memory=True, persistent_workers=True)

    lat = np.zeros((len(ds), latent_dim), dtype=np.float32)
    cur = 0
    with torch.inference_mode():
        for flux, mask, meta in tqdm(dl, desc=f"Encoder({ids_npy.stem})"):
            lat[cur:cur+flux.size(0)] = model.encoder(flux.cuda())[0].cpu().numpy()
            cur += flux.size(0)

    np.save(out_npy, lat)
    return lat

In [8]:
X_train = extract_latent(TRAIN_IDS_NPY, LAT_TRAIN_NPY)
X_val   = extract_latent(VAL_IDS_NPY,   LAT_VAL_NPY)

✓ latent_train.npy ya existe → cargando
✓ latent_val.npy ya existe → cargando


In [9]:
   # 2) Estandarizar con stats del train
scaler  = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std   = scaler.transform(X_val)

In [10]:
# 3) Etiquetas proxy raras en validation
meta_df   = pd.read_csv("../data/raw/plasticc_train_metadata.csv", usecols=["object_id", "target"])
freqs     = meta_df["target"].value_counts().sort_values()
rare_set  = set(freqs.index[:PROXY_RARE])
val_ids   = np.load(VAL_IDS_NPY)
meta_dict = dict(zip(meta_df["object_id"].values, meta_df["target"].values))
y_true_val = np.array([1 if meta_dict.get(int(i), -1) in rare_set else 0 for i in val_ids])

In [ ]:
 
GRID_PARAMS     = {
    "n_estimators":  [100, 200, 300],
    "max_samples":   ["auto", 0.5, 0.7, 0.9],
    "contamination": [0.01, 0.03, 0.05, 0.08, 0.1],
    "max_features":  [0.5, 0.7, 0.9, 1.0],
    "bootstrap":     [True, False],
}

In [ ]:
# 4) Grid-search Isolation Forest
best_auc, best_iso, best_params = -1, None, None

# Calcular total de combinaciones para mostrar progreso
total_combinations = len(GRID_PARAMS["n_estimators"]) * len(GRID_PARAMS["max_samples"]) * \
                    len(GRID_PARAMS["contamination"]) * len(GRID_PARAMS["max_features"]) * \
                    len(GRID_PARAMS["bootstrap"])

print(f"Iniciando grid search con {total_combinations} combinaciones...")

for i, (n_estim, max_samp, contam, max_feat, bootstrap) in enumerate(
    tqdm(product(GRID_PARAMS["n_estimators"],
                 GRID_PARAMS["max_samples"],
                 GRID_PARAMS["contamination"],
                 GRID_PARAMS["max_features"],
                 GRID_PARAMS["bootstrap"]), 
         total=total_combinations,
         desc="Grid Search")):
    
    iso = IsolationForest(
        n_estimators=n_estim, max_samples=max_samp,
        contamination=contam, max_features=max_feat, bootstrap=bootstrap,
        random_state=42, n_jobs=-1
    ).fit(X_train_std)

    scores_val = -iso.score_samples(X_val_std)  # signo invertido (más grande = outlier)
    auc = roc_auc_score(y_true_val, scores_val)

    # print(f"IF {n_estim=} {max_samp=} {contam=} {max_feat=} {bootstrap=} → AUC={auc:.3f}")
    if auc > best_auc:
        best_auc, best_iso, best_params = auc, iso, (n_estim, max_samp, contam, max_feat, bootstrap)

print(f"\n🏆 Mejor AUC={best_auc:.3f} con params {best_params}")

Iniciando grid search con 480 combinaciones...


Grid Search: 100%|██████████| 480/480 [28:34<00:00,  3.57s/it]   


🏆 Mejor AUC=0.472 con params (100, 0.9, 0.01, 1.0, True)


In [13]:
# 5) Umbral = varios percentiles de score sobre train
scores_train = -best_iso.score_samples(X_train_std)  # signo invertido para consistencia
thresholds = {
    "score_threshold_p1": float(np.percentile(scores_train, 1)),    # percentil 1%
    "score_threshold_p3": float(np.percentile(scores_train, 3)),    # percentil 3%
    "score_threshold_p5": float(np.percentile(scores_train, 5)),    # percentil 5%
    "score_threshold_p10": float(np.percentile(scores_train, 10)),  # percentil 10%
}

print("Umbrales calculados:")
for key, value in thresholds.items():
    print(f"{key} = {value:.4f}")

# Mantener thr_p5 para compatibilidad con el código siguiente
thr_p5 = thresholds["score_threshold_p5"]

Umbrales calculados:
score_threshold_p1 = 0.3034
score_threshold_p3 = 0.3035
score_threshold_p5 = 0.3035
score_threshold_p10 = 0.3036


In [14]:
# 6) Guardar modelo + scaler + umbral
MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump({"iso": best_iso, "scaler": scaler}, MODEL_OUT)
with open(THRESH_JSON, "w") as f:
    json.dump(thresholds, f, indent=4)
print(f"✅ Modelo IsolationForest guardado en {MODEL_OUT}")
print(f"✅ Umbral guardado en {THRESH_JSON}")

✅ Modelo IsolationForest guardado en c:\Users\David\___utad\4_Cuarto\TFG\proyecto_PLAsTiCC\models\iso_model.pkl
✅ Umbral guardado en c:\Users\David\___utad\4_Cuarto\TFG\proyecto_PLAsTiCC\models\threshold_iso.json
